In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

In [2]:
b = CF( (y-0.5, 0.5-x) )
Draw (b, mesh, "wind", vectors={"grid_size" : 20 });

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [3]:
fes = L2(mesh, order=3)

u, v = fes.TnT()
a = BilinearForm(fes, nonassemble=True)

a += -b*u*grad(v)*dx

# the upwind-term:
n = specialcf.normal(2)
uup = IfPos(b*n, u, u.Other(bnd=0))
a += b*n*uup*v * dx(element_boundary=True)

#f = LinearForm(fes).Assemble()

gfu = GridFunction(fes)
gfu.Set(exp(-10**2*((x-0.5)*(x-0.5) +(y-0.75)*(y-0.75))))

scene = Draw(gfu, min=0, max=1, order=3, autoscale=False)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [4]:
tau = 0.001
tend = 50
t = 0
cnt = 0

w = gfu.vec.CreateVector()
# the mass matrix (implemented matrix-free)
invm = fes.Mass(rho=1).Inverse()
SetNumThreads(3)
with TaskManager():
    while t < tend:
        # print (t)
        # apply the transport operator 
        a.Apply (gfu.vec, w)
        gfu.vec.data -= tau * invm * w
        if cnt%20 == 0:
            scene.Redraw()    
        t += tau
        cnt +=1 